![Logo TUBAF](https://tu-freiberg.de/sites/default/files/media/freiberger-alumni-netzwerk-6127/wbm_orig_rgb_0.jpg)

In our teaching we employ interactive Jupyter notebooks in order to allow our students to play with the concepts and hence familiarize themselves more quickly with the new knowledge. Below, we give two simple examples of such scripts. The remainder can be found in Opal in the respective course materials.

Comments to:

*Prof. Dr. Thomas Nagel  
Chair of Soil Mechanics and Foundation Engineering  
Geotechnical Institute  
Technische Universität Bergakademie Freiberg.*

https://tu-freiberg.de/en/soilmechanics

# Beschreibung von Kornverteilungskurven

In [11]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import widgets
from ipywidgets import interact
from ipywidgets import interaction
import scipy as sp
from scipy import interpolate
import pandas as pd

#Some plot settings
import plot_functions.plot_settings
%run plot_functions/kornverteilungskurve_plots.ipynb

Im Zuge der Bestimmung einer Sieblinie werden Siebrückstände $m_i$ ermittelt. Die einzelnen zu den Siebdurchmessern $d_i$ gehörigen Siebrückstände $m_i$ können nun auf die Gesamtmasse $m_\text{d} = \sum m_i$ (Annahme vernachlässigbarer Siebverluste) bezogen werden, womit die Masseanteile

$$
    x_i = \frac{m_i}{m_\text{d}}
$$

ermittelt werden. Diese werden typischerweise in einer Summenkurve

$$
    F(d_i) = \sum \limits_{k=1}^i x_i
$$

aufgetragen. Dabei wird aufgrund der über Größenordnungen verteilten Korndurchmesser eine halblogarithmische Darstellung gewählt (siehe folgende Abbildung, rechts). Feinanteile, die nicht durch Siebung näher untersucht werden, sind nicht mit dargestellt. In der linken Abbildung sind die Masseanteile selbst und das zugehörige Histogramm der Masseverteilung dargestellt.

Mithilfe der Körnungslinie bzw. Kornverteilungskurve können die dominierenden Bestandteile (T, U, S, G) ermittelt werden. Des weiteren können die Ungleichförmigkeitszahl

$$
    C_U = \frac{d_{60}}{d_{10}}
$$

und die Krümmungszahl

$$
    C_C = \frac{d_{30}^2}{d_{10}\ d_{60}}
$$

abgeleitet werden. 

Die folgenden Siebrückstände seien gegeben:

In [12]:
Siebdurchmesser = np.array([63,31.5,16,8,4,2,1,0.5,0.25,0.125,0.063,0.001])#letzter: Rückstand in Null
Startwerte = np.array([0,0,842.4,1059.8,1222.9,788.0,706.6,407.6,210.0,195.7,0,0])

Massen = [widgets.FloatText(value=Startwerte[i],description=str(Siebdurchmesser[i])+' mm') for i in range(len(Siebdurchmesser))]
Eingabe=widgets.VBox(Massen)
Eingabe

In [13]:
def plot(Neuauswertung):
    m_i = Siebmassen(Eingabe)
    dm_i = Masseanteile(m_i)
    sum_dm_i = Siebdurchgang(dm_i)
    plot_KVK(sum_dm_i)

interact(plot, Neuauswertung=True);

interactive(children=(Checkbox(value=True, description='Neuauswertung'), Output()), _dom_classes=('widget-inte…

Damit lassen sich die Masseanteile der einzelnen Kornfraktionen ablesen:

In [14]:
def Anteilsbestimmung(Neuauswertung):
    a=Anteile(Siebdurchmesser,Siebdurchgang(Masseanteile(Siebmassen(Eingabe))))
    display(a)
    
interact(Anteilsbestimmung, Neuauswertung=True);

interactive(children=(Checkbox(value=True, description='Neuauswertung'), Output()), _dom_classes=('widget-inte…

# Schranken für $K_0$ aus dem Mohr'schen Spannungskreis

In [15]:
from matplotlib import patches

#Some plot settings
%run plot_functions/Mohrs_circle_K0_plots.ipynb

## Ebene Betrachtungen

Die folgenden Betrachtungen liegen in der $x$-$z$ Ebene und gelten für ebene Spannungszustände und ebene Verzerrungszustände mit $\sigma_{yy} = \sigma_2$.

Zusätzlich legen wir $\sigma_{zz}$ beliebig, aber ohne Einschränkung der Allgemeingültigkeit, zu 200 kPa fest.

Im kohäsionslosen Boden ergibt sich aus dem Mohr-Coulomb'schen Bruchkriterium:

$$
\displaystyle\frac{1-\sin\phi}{1+\sin \phi} \sigma_{zz} < \sigma_{xx} < \frac{1+\sin\phi}{1-\sin \phi} \sigma_{zz}
$$

mit 

\begin{align*}
    K_{0\text{a}} &= \displaystyle\frac{1-\sin\phi}{1+\sin \phi}
    \\\\
    K_{0\text{p}} &= \displaystyle \frac{1+\sin\phi}{1-\sin \phi}
\end{align*}

In [16]:
s_zz = 200.

In [17]:
plot_interactive_K0()

interactive(children=(FloatSlider(value=30.0, description='$\\phi$ / °', max=45.0, min=10.0, step=5.0), FloatS…

In kohäsiven Böden ergibt sich in analoger Betrachtungsweise der von den aktiven und passiven Grenzzuständen begrenzte Horizontalspannungsbereich

$$
K_{0\text{a}} \sigma_{zz} - 2c \sqrt{K_{0\text{a}}} < \sigma_{xx} < K_{0\text{p}} \sigma_{zz} + 2c \sqrt{K_{0\text{p}}}
$$

In [18]:
plot_interactive_K0c()

interactive(children=(FloatSlider(value=20.0, description='$\\phi$ / °', max=30.0, min=10.0, step=5.0), FloatS…